In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re

ruta_archivo = "C:/Users/Usuario/Documents/Henry/PI/Dataset/movies_dataset.csv"
dataset = pd.read_csv(ruta_archivo)

# Cambiar los valores nulos por 0
dataset['revenue'] = np.nan_to_num(dataset['revenue'].fillna(0))
dataset['budget'] = np.nan_to_num(dataset['budget'].fillna(0))

# Desanidar los campos anidados
# Belongs to collection
dataset['belongs_to_collection'] = dataset['belongs_to_collection'].str.replace("'", '"')
def extract_name(x):
    try:
        data = json.loads(x)
        return data['name'] if 'name' in data else None
    except (json.JSONDecodeError, TypeError):
        return None

dataset['belongs_to_collection'] = dataset['belongs_to_collection'].apply(lambda x: extract_name(x) if pd.notnull(x) and isinstance(x, str) else None)

# Producion companies
def extract_production_company_name(row):
    try:
        companies = ast.literal_eval(row)
        if isinstance(companies, list):
            if len(companies) > 0:
                return ', '.join([company['name'] for company in companies])
        elif isinstance(companies, dict):
            return companies['name']
    except (ValueError, SyntaxError):
        pass
    return None

# Aplicar la función de extracción a la columna "production_companies"
dataset['production_companies'] = dataset['production_companies'].apply(extract_production_company_name)

# Genres
# Comprobamos si la columna "genres" contiene valores nulos
if dataset['genres'].isnull().any():
    # En caso de haber valores nulos, los rellenamos con una cadena vacía
    dataset['genres'] = dataset['genres'].fillna('')

# Convertimos la columna "genres" en formato de cadena
dataset['genres'] = dataset['genres'].astype(str)

# Función para extraer los nombres de los géneros
def extract_genre_names(genres):
    try:
        genre_list = json.loads(genres.replace("'", '"'))
        genre_names = [genre['name'] for genre in genre_list]
        return ', '.join(genre_names)
    except:
        return ''

# Aplicamos la función para extraer los nombres de los géneros en una nueva columna
dataset['genres'] = dataset['genres'].apply(extract_genre_names)



# Production countries
if dataset['production_countries'].isnull().any():
    # En caso de haber valores nulos, los rellenamos con una cadena vacía
    dataset['production_countries'] = dataset['production_countries'].fillna('')

# Convertimos la columna "production_countries" en formato de cadena
dataset['production_countries'] = dataset['production_countries'].astype(str)

# Función para extraer los nombres de los países de producción
def extract_country_names(countries):
    try:
        country_list = json.loads(countries.replace("'", '"'))
        country_names = [country['name'] for country in country_list]
        return ', '.join(country_names)
    except:
        return ''

# Aplicamos la función para extraer los nombres de los países de producción en una nueva columna
dataset['production_countries'] = dataset['production_countries'].apply(extract_country_names)

# Spoken Languages
# Comprobamos si la columna "spoken_languages" contiene valores nulos
if dataset['spoken_languages'].isnull().any():
    # En caso de haber valores nulos, los rellenamos con una cadena vacía
    dataset['spoken_languages'] = dataset['spoken_languages'].fillna('')

# Convertimos la columna "spoken_languages" en formato de cadena
dataset['spoken_languages'] = dataset['spoken_languages'].astype(str)

# Función para extraer los nombres de los idiomas
def extract_language_names(languages):
    try:
        language_list = json.loads(languages.replace("'", '"'))
        language_names = [language['name'] for language in language_list]
        return ', '.join(language_names)
    except:
        return ''

# Aplicamos la función para extraer los nombres de los idiomas en una nueva columna
dataset['spoken_languages'] = dataset['spoken_languages'].apply(extract_language_names)

# Eliminar los valores nulos de la tabla release date
dataset = dataset.dropna(subset=['release_date'])

# Crear la columna "release_year" con el año de la fecha de estreno
dataset['release_date'] = pd.to_datetime(dataset['release_date'], format='%Y-%m-%d', errors='coerce')
dataset['release_year'] = dataset['release_date'].dt.year

# Creando la columna return
dataset['budget'] = dataset['budget'].apply(lambda x: re.sub('[^\d.]', '', str(x)))

dataset['revenue'] = dataset['revenue'].astype(float)
dataset['budget'] = dataset['budget'].astype(float)
dataset['return'] = dataset['revenue'] / dataset['budget']
dataset['return'].fillna(0, inplace=True)


# Eliminando las columnas pedidas
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
dataset.drop(columns_to_drop, axis=1, inplace=True)

dataset.head()

ruta_archivo_modificado = "C:/Users/Usuario/Documents/Henry/PI/Dataset/movies_dataset_modificado.csv"
dataset.to_csv(ruta_archivo_modificado, index=False)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_1964\79359772.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(ruta_archivo)
